# Creating train-val split


In [5]:
## importing required libraries
import os
import shutil
import random

!pip install ultralytics --quiet
!pip install tqdm --upgrade --quiet
from tqdm.notebook import tqdm


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
print("pwd: ", os.getcwd())
train_path_img = "../vegnet_images/train/"
train_path_label = "../vegnet_annotations/vegnet-annots-train-updated.csv"
val_path_img = "../vegnet_images/val/"
val_path_label = "../vegnet_annotations/vegnet-annots-val-updated.csv"
test_path = "../vegnet_images/vis_test_val"

pwd:  /home/s/repo/vegnet-disease-det/yolo


In [ ]:
"""
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
"""


def train_test_split(path, neg_path=None, split=0.2):
    print("------ PROCESS STARTED -------")

    files = list(
        set([name[:-4] for name in os.listdir(path)])
    )  ## removing duplicate names i.e. counting only number of images

    print(f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok=True)
    os.makedirs(train_path_label, exist_ok=True)
    os.makedirs(val_path_img, exist_ok=True)
    os.makedirs(val_path_label, exist_ok=True)

    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
        if filex == "classes":
            continue
        shutil.copy2(path + filex + ".jpg", f"{train_path_img}/" + filex + ".jpg")
        shutil.copy2(path + filex + ".txt", f"{train_path_label}/" + filex + ".txt")

    print(
        f"------ Training data created with 80% split {len(files[:train_size])} images -------"
    )

    if neg_path:
        neg_images = list(
            set([name[:-4] for name in os.listdir(neg_path)])
        )  ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(
                neg_path + filex + ".jpg", f"{train_path_img}/" + filex + ".jpg"
            )

        print(
            f"------ Total  {len(neg_images)} negative images added to the training data -------"
        )

        print(
            f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------"
        )

    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
        if filex == "classes":
            continue
        # print("running")
        shutil.copy2(path + filex + ".jpg", f"{val_path_img}/" + filex + ".jpg")
        shutil.copy2(path + filex + ".txt", f"{val_path_label}/" + filex + ".txt")

    print(
        f"------ Testing data created with a total of {len(files[train_size:])} images ----------"
    )

    print("------ TASK COMPLETED -------")


## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
# train_test_split('/content/drive/MyDrive/yolov8/data/') ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images

In [1]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.220 🚀 Python-3.10.11 torch-2.1.0.dev20230503+cu117 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)
Setup complete ✅ (16 CPUs, 15.9 GB RAM, 324.2/465.5 GB disk)


## Training


In [8]:
# !yolo task=detect mode=train model=yolov8s.pt data=vegnet_yolo.yaml epochs=10 imgsz=512 batch=8 project=../yolo_out name=vegnet_yolov8s_augmented_100
!yolo task=detect mode=train model=../yolo_out/vegnet_yolov8s_200_210_untrained/weights/best.pt \
    data="vegnet_yolo.yaml" epochs=200 batch=8 imgsz=256 \
        device=0 workers=8 optimizer=Adam pretrained=false \
            val=true plots=true save=True save_period=5	show=true optimize=true patience=50 \
                lr0=0.00001 lrf=0.001 fliplr=0.0 amp=False name=vegnet_yolov8s_200_210_untrained_cont \
                    project=../yolo_out


New https://pypi.org/project/ultralytics/8.0.222 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.218 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce GTX 1650 with Max-Q Design, 3904MiB)
engine/trainer: task=detect, mode=train, model=../yolo_out/vegnet_yolov8s_200_210_untrained/weights/best.pt, data=vegnet_yolo.yaml, epochs=200, patience=50, batch=8, imgsz=256, save=True, save_period=5, cache=False, device=0, workers=8, project=../yolo_out, name=vegnet_yolov8s_200_210_untrained_cont, exist_ok=False, pretrained=False, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnost


     23/100      4.44G      1.101       1.32       1.48         34        512:  23%|██▎       | 65/279 [00:10<00:35,  6.06it/s]
     23/100      4.44G      1.103      1.319       1.48         25        512:  23%|██▎       | 65/279 [00:10<00:35,  6.06it/s]
     23/100      4.44G      1.103      1.319       1.48         25        512:  24%|██▎       | 66/279 [00:10<00:34,  6.11it/s]
     23/100      4.44G      1.107      1.324      1.483         16        512:  24%|██▎       | 66/279 [00:11<00:34,  6.11it/s]
     23/100      4.44G      1.107      1.324      1.483         16        512:  24%|██▍       | 67/279 [00:11<00:34,  6.13it/s]
     23/100      4.44G      1.109      1.324      1.485         32        512:  24%|██▍       | 67/279 [00:11<00:34,  6.13it/s]
     23/100      4.44G      1.109      1.324      1.485         32        512:  24%|██▍       | 68/279 [00:11<00:34,  6.15it/s]
     23/100      4.44G      1.107      1.328      1.484         15        512:  24%|██▍       | 68/279 

## Inferencing


In [11]:
!yolo task=detect mode=predict model=../yolo_out/vegnet_yolov8s_original_dataset/weights/best.pt conf=0.55 source=../datasets/vegnet/vis_test_val

Ultralytics YOLOv8.0.218 🚀 Python-3.11.4 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce GTX 1650 with Max-Q Design, 3904MiB)
Model summary (fused): 168 layers, 11126745 parameters, 0 gradients, 28.4 GFLOPs

image 1/4 /home/s/repo/vegnet-disease-det/yolo/../datasets/vegnet/vis_test_val/Bacterial Spot. (149).jpg: 384x512 3 Bacterial Spot Rots, 62.8ms
image 2/4 /home/s/repo/vegnet-disease-det/yolo/../datasets/vegnet/vis_test_val/Black Rot(13).jpeg: 384x512 1 Black Rot, 12.0ms
image 3/4 /home/s/repo/vegnet-disease-det/yolo/../datasets/vegnet/vis_test_val/Black Rot(13).jpg: 384x512 1 Black Rot, 12.1ms
image 4/4 /home/s/repo/vegnet-disease-det/yolo/../datasets/vegnet/vis_test_val/No disease. (103).jpg: 512x384 (no detections), 56.6ms
Speed: 2.1ms preprocess, 35.9ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 384)
Results saved to /home/s/repo/vegnet-disease-det/runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/yolov8/output

### inferencing on video


In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8/training_results/football/weights/best.pt conf=0.77 source=/content/drive/MyDrive/yolov8/vid2

In [ ]:
!cp -r /content/runs/detect/predict2 /content/drive/MyDrive/yolov8/output

In [ ]:
# !yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8/training_results/football2/weights/best.pt conf=0.5 source=/content/drive/MyDrive/yolov8/videos

In [ ]:
# !cp -r /content/runs/detect/predict3 /content/drive/MyDrive/yolov8/output

## Export the model -- In progress

Export a YOLOv8 model to any supported format with the `format` argument, i.e. `format=onnx`.

| Format                | `format=...**bold text**` | Model                   |
| --------------------- | ------------------------- | ----------------------- |
| PyTorch               | -                         | yolov8n.pt              |
| TorchScript           | `torchscript`             | yolov8n.torchscript     |
| ONNX                  | `onnx`                    | yolov8n.onnx            |
| OpenVINO              | `openvino`                | yolov8n_openvino_model/ |
| TensorRT              | `engine`                  | yolov8n.engine          |
| CoreML                | `coreml`                  | yolov8n.mlmodel         |
| TensorFlow SavedModel | `saved_model`             | yolov8n_saved_model/    |
| TensorFlow GraphDef   | `pb`                      | yolov8n.pb              |
| TensorFlow Lite       | `tflite`                  | yolov8n.tflite          |
| TensorFlow Edge TPU   | `edgetpu`                 | yolov8n_edgetpu.tflite  |
| TensorFlow.js         | `tfjs`                    | yolov8n_web_model/      |
| PaddlePaddle          | `paddle`                  | yolov8n_paddle_model/   |


In [ ]:
!yolo mode=export model=/content/drive/MyDrive/yolov8/training_results/football2/weights/best.pt format=onnx